In [1]:

%load_ext autoreload
%autoreload 2
import tqdm
import numpy as np
from numpy import linalg as la
from sklearn import svm

from functions import tda
from functions import pwk  # persistence weighted kernel
import utils

In [2]:
x_train, y_train, x_test, y_test = utils.get_data_images('mnist')

Loaded persistence diagrams.


In [3]:
def nearestPD(A):
    """
    Adapted from @https://stackoverflow.com/questions/43238173/python-convert-matrix-to-positive-semi-definite
    Find the nearest positive-definite matrix to input
    
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].

    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False


In [4]:
def get_gram(list_pd):
    def function_weight(_name_weight, _val_c=1.0, _val_p=1):
        if _name_weight == "arctan":
            def _func_weight(vec_bd):
                return np.arctan(
                    np.power((vec_bd[1] - vec_bd[0]) / _val_c, _val_p))
        else:  # p-persistence
            def _func_weight(vec_bd):
                return np.power(vec_bd[1] - vec_bd[0], _val_p)
        return _func_weight

    def function_kernel(_name_kernel, _val_sigma=1.0):
        if _name_kernel == "Gaussian":
            def _func_kernel(vec_bd_1, vec_bd_2):
                val_dist = np.power(np.linalg.norm(vec_bd_1 - vec_bd_2, 2), 2)
                return np.exp(-1.0 * val_dist / (2.0 * np.power(val_sigma, 2)))
        else:  # linear kernel
            def _func_kernel(vec_bd_1, vec_bd_2):
                return np.dot(vec_bd_1, vec_bd_2)
        return _func_kernel

    val_sigma = tda.parameter_sigma(list_pd)
    val_c = tda.parameter_birth_death_pers(list_pd)[2]
    val_p = 5
    func_kernel = function_kernel("Gaussian", val_sigma)
    func_weight = function_weight("arctan", val_c, val_p)

    name_rkhs = ["Gaussian", "Linear"][1]
    approx = [True, False][0]
    class_pwk = pwk.Kernel(
        list_pd, func_kernel, func_weight, val_sigma, name_rkhs, approx)
    mat_gram = class_pwk.gram()
    return mat_gram

In [5]:
def process_data(x):
    '''
        Computes the gram matrix for the cubical filtration
        We restrict to the cubical filtration because it becomes expensive in both cpu and memory to do all filtrations.
        If needed other filtrationed could be individually tested, but when alone cubical filtration gives the best results.
    '''
    N = x[0].shape[0]
    grams = []
    
    for xtf in x[:1]: # Get cubical filtration
        list_pds = []
        i=0
        for pd_img in xtf[:1000]:
            n = pd_img.shape[1]

            #flatten accross axis=0 
            pd_img = np.reshape(pd_img,newshape=(2*n,2))
            
            # Remove zeros
            mask1 = pd_img[:,0]==0
            mask2 = pd_img[:,1]==0
            mask=mask1*mask2
            pd_img = pd_img[~mask,:]
            
            # Add pd to list
            list_pds.append(np.sqrt(pd_img))
            
        # Get gram matrix for current filtration
        gr = get_gram(list_pds)
        gr = nearestPD(gr)
        
        # Append it to the list
        grams.append(gr)
    return grams

In [7]:
import datetime
begin_time = datetime.datetime.now()
K_train = process_data(x_train)
K_test = process_data(x_test)
end_time = datetime.datetime.now()
print(datetime.datetime.now() - begin_time)


0:00:07.873235


In [ ]:
clf = svm.SVC(kernel='precomputed')
clf.fit(K_train[0],y_train[:1000])

In [ ]:
clf.score(K_test[0],y_test[:1000])